In [1]:
import pandas as pd
import re

In [2]:

def preprocess_names(df, Column_Name):
    #Remove space if the first letter of the word is a space
    df['temp_Column_Name'] = df[Column_Name]
    df['temp_Column_Name'] = df['temp_Column_Name'].str.lstrip()
    
    #Remove single letter in the Names
    #df[Column_Name] = df[Column_Name].str.replace(r'\b[A-Za-z]\b', '')
    df['temp_Column_Name'] = df['temp_Column_Name'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 1]))
    #Remove names in brackets
    #df[Column_Name] = df[Column_Name].str.replace(r'\s?\([^()]*\)', '')
    df['temp_Column_Name'] = df['temp_Column_Name'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
    
    rwords = ['Doctor','Professor','Captain','Jr','Sr','III','II', 'IV']
    for word in rwords:
        df.loc[:, 'temp_Column_Name'] = df['temp_Column_Name'].str.replace(word,"")
        
    df['temp_Column_Name'] = df['temp_Column_Name'].str.lstrip()
    
    df['First Name'] = df['temp_Column_Name'].str.split(" ", expand=True)[0]
    df['Last Name'] = df['temp_Column_Name'].str.split().str[-1]
    df.drop(columns=['temp_Column_Name'], inplace=True)
    return df

In [3]:
dx = pd.read_excel('Master List - position types.xlsx')

In [4]:
dx['Director Name']

0           Michael (Mike) B Tully
1                  Mary R Hennessy
2        James (JK) Kevin Symancyk
3            Kenneth (Ken) C Hicks
4                     Anita Sehgal
                   ...            
14984                    Eric Liaw
14985                    Eric Liaw
14986                   Ian Siegel
14987                   Yvonne Hao
14988         David (Dave) Travers
Name: Director Name, Length: 14989, dtype: object

In [5]:
dg = preprocess_names(dx, 'Director Name')

In [6]:
dg.shape

(14989, 9)

In [7]:
import gender_guesser.detector as gender

d = gender.Detector(case_sensitive=False)
dg['gender'] = dg['First Name'].apply(lambda x : d.get_gender(x))

In [8]:
dg.shape

(14989, 10)

In [9]:
dx = dg.copy()

In [10]:
from ethnicolr import pred_fl_reg_name

eth = pred_fl_reg_name(dx,'Last Name', 'First Name')

In [12]:
print(eth.shape, dx.shape)

(6360, 15) (6360, 11)


In [33]:
dg1 = dg.copy()

In [34]:
dg1.columns

Index(['Company Name', 'Director Name', 'id (boardex)', 'Role',
       'Role Description (if applicable)', 'Start Date', 'End Date',
       'First Name', 'Last Name', 'gender'],
      dtype='object')

In [35]:
eth.columns

Index(['Company Name', 'Director Name', 'id (boardex)', 'Role',
       'Role Description (if applicable)', 'Start Date', 'End Date',
       'First Name', 'Last Name', 'gender', 'asian', 'hispanic', 'nh_black',
       'nh_white', 'race'],
      dtype='object')

In [36]:
dg1 = dg1.merge(eth, on=[
       'First Name', 'Last Name'])

In [41]:
full_df = dg1.copy()

In [42]:
full_df = full_df[['Company Name_x', 'Director Name_x', 'id (boardex)_x', 'Role_x',
       'Role Description (if applicable)_x', 'Start Date_x', 'End Date_x',
       'First Name', 'Last Name', 'gender_x','asian', 'hispanic', 'nh_black', 'nh_white', 'race']]

In [43]:
full_df = full_df.rename(columns={
    'Company Name_x': 'Company Name', 
    'Director Name_x':'Director Name', 
    'id (boardex)_x':'id (boardex)', 
    'Role_x':'Role',
    'Role Description (if applicable)_x':'Role Description (if applicable)',
    'Start Date_x':'Start Date', 
    'End Date_x':'End Date',
    'gender_x':'gender'
})

In [44]:
full_df.to_csv('')

,Company Name,Director Name,id (boardex),Role,Role Description (if applicable),Start Date,End Date,First Name,Last Name,gender,asian,hispanic,nh_black,nh_white,race
0,AAA Life Insurance Co,Michael (Mike) B Tully,22555745259,Director - SD,NaN,1998,2017-07-01 00:00:00,Michael,Tully,male,0.002142,0.010533,0.104182,0.883143,nh_white
1,AAA Life Insurance Co,Mary R Hennessy,3671227737,Chairman,NaN,2018-09-01 00:00:00,Current,Mary,Hennessy,mostly_female,0.002049,0.007915,0.046994,0.943042,nh_white
2,Academy Ltd (Academy Sports + Outdoors),James (JK) Kevin Symancyk,15201918545,President/CEO,NaN,2015-11-02 00:00:00,2018-05-16 00:00:00,James,Symancyk,male,0.001452,0.003475,0.019194,0.975880,nh_white
3,Meijer Inc,James (JK) Kevin Symancyk,15201918545,President,NaN,2013-02-03 00:00:00,2015-10-01 00:00:00,James,Symancyk,male,0.001452,0.003475,0.019194,0.975880,nh_white
4,Meijer Inc,James (JK) Kevin Symancyk,15201918545,Group VP,Perishables,2006-10-01 00:00:00,2007,James,Symancyk,male,0.001452,0.003475,0.019194,0.975880,nh_white
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14984,ZIPRECRUITER INC,Cipora Herman,187220611506,Lead Independent Director,NaN,2021-05-26 00:00:00,Current,Cipora,Herman,unknown,0.103955,0.117007,0.091652,0.687386,nh_white
14985,ZIPRECRUITER INC,Emily McEvilly,22540585244,Independent Director,NaN,2022-09-01 00:00:00,Current,Emily,McEvilly,female,0.003221,0.017185,0.020939,0.958656,nh_white
14986,ZIPRECRUITER INC,Eric Liaw,81185810522,Independent Director,NaN,2021-05-26 00:00:00,Current,Eric,Liaw,male,0.013964,0.021110,0.087865,0.877061,nh_white
14987,ZIPRECRUITER INC,Eric Liaw,81185810522,Independent Director,NaN,2021-05-26 00:00:00,Current,Eric,Liaw,male,0.013964,0.021110,0.087865,0.877061,nh_white


In [45]:
full_df.to_csv('Master_List_with_Ethinicity_and_gender.csv')